In [ ]:
pip install scikit-learn numpy dlib opencv-python imutils matplotlib scipy cmake dlib


In [4]:
from sklearn.cluster import KMeans
import numpy as np, dlib, cv2, imutils, time, timeit
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread

import sys
sys.path.append('../processing')

from removeLight import remove_light

In [6]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [8]:
def calculate_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [ ]:
def initialize_adaptive_ear(vs, frame_count=5):
    def measure_ear_state(is_eye_open):
        state = "뜸" if is_eye_open else "감음"
        print(f"[INFO] 눈 {state} 상태 측정을 시작")
        print(f"[INFO] 측정 중... ({state})")

        ears = []
        collected = 0
        while collected < frame_count:
            frame = vs.read()
            frame = imutils.resize(frame, width=400)
            _, gray = remove_light(frame)
            rects = detector(gray, 0)

            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = calculate_ear(leftEye)
                rightEAR = calculate_ear(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                ears.append(ear)
                collected += 1
                print(f"[DEBUG] EAR ({state}): {ear:.3f}")
                break  # 한 명만 인식

            time.sleep(0.05)

        return sum(ears) / len(ears)

    open_ear = measure_ear_state(is_eye_open=True)
    close_ear = measure_ear_state(is_eye_open=False)

    threshold = ((open_ear - close_ear) / 2.0) + close_ear
    print(f"[INFO] 자동 설정된 EAR 기준값: {threshold:.3f} (OPEN: {open_ear:.3f}, CLOSE: {close_ear:.3f})\n")
    return threshold


In [17]:
def is_eye_closed(frame, threshold):
    frame = imutils.resize(frame, width=400)
    _, gray = remove_light(frame)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = calculate_ear(leftEye)
        rightEAR = calculate_ear(rightEye)
        both_ear = (leftEAR + rightEAR) / 2.0
        print(f"[DEBUG] EAR: {both_ear:.3f}, Threshold: {threshold:.3f}")
        return both_ear < threshold

    return False

In [ ]:
vs = VideoStream(src=0).start()
time.sleep(1.0)

EAR_THRESH = initialize_adaptive_ear(vs)

while True:
    frame = vs.read()
    if is_eye_closed(frame, EAR_THRESH):
        print("눈 감음")
    else:
        print("눈 뜸")

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

[INFO] 눈 뜸 상태 측정을 시작합니다.
[INFO] 측정 중... (뜸)
[DEBUG] EAR (뜸): 0.200
[DEBUG] EAR (뜸): 0.220
[DEBUG] EAR (뜸): 0.206
[DEBUG] EAR (뜸): 0.191
[DEBUG] EAR (뜸): 0.197
[INFO] 눈 감음 상태 측정을 시작합니다.
[INFO] 측정 중... (감음)
[DEBUG] EAR (감음): 0.203
[DEBUG] EAR (감음): 0.230
[DEBUG] EAR (감음): 0.190
[DEBUG] EAR (감음): 0.208
[DEBUG] EAR (감음): 0.196
[INFO] 자동 설정된 EAR 기준값: 0.204 (OPEN: 0.203, CLOSE: 0.205)

[DEBUG] EAR: 0.236, Threshold: 0.204
눈 뜸
[DEBUG] EAR: 0.195, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.190, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.197, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.179, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.198, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.197, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.195, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.209, Threshold: 0.204
눈 뜸
[DEBUG] EAR: 0.195, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.199, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.193, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.187, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.197, Threshold: 0.204
눈 감음
[DEBUG] EAR: 0.197, Threshold: 

KeyboardInterrupt: 